In [1]:
import sys
sys.path.insert(0, '../../vector_field/velvet_ecosystem/velvet/')
import velvet as vt

import scvelo as scv
import scanpy as sc

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm, trange



/camp/home/maizelr/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /camp/home/maizelr/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")
Global seed set to 0
/camp/home/maizelr/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/camp/home/maizelr/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_exp

In [2]:
adata0 = vt.pp.read("../data/celltyped_adata_splicing.h5ad")
adata0 = adata0[adata0.obs.cell_annotation!='other']

In [3]:
adata0

View of AnnData object with n_obs × n_vars = 45037 × 24967
    obs: 'X_TC_alpha', 'total_TC_alpha', 'unspliced_TC_alpha', 'spliced_TC_alpha', 'p_e', 'p_c_TC', 'sample', 'timepoint', 'rep', 'marker_based_classification', 'cell_annotation', 'cell_annotation_detailed'
    layers: 'spliced', 'unspliced'

In [4]:
def process_splicing(adata0, reference_name):
    """
    load the labelling data, subset splicing data to same cells and genes,
    """
    assert 'spliced' in adata0.layers.keys(), 'this data has no splicing information'

    ref = sc.read_h5ad(f'../data/benchmarking_data/{reference_name}.h5ad')
    
    adata = adata0[ref.obs_names, ref.var_names].copy()
    adata.obs = ref.obs
    adata.var = ref.var
    
    # this ensures we can map velocity to same space as benchmark data:
    adata.layers['total'] = ref.layers['total'].copy()
    adata.obsm['X_pca'] = ref.obsm['X_pca'].copy()
    adata.obsm['velocity_cr_pca'] = ref.obsm['velocity_cr_pca']
    if 'maxi' not in reference_name:
        adata.obsm['velocity_pst'] = ref.obsm['velocity_pst']
    
    
    sc.pp.normalize_total(adata, layers=['spliced'], target_sum=None)
    sc.pp.normalize_total(adata, layers=['unspliced'], target_sum=None)
    
    adata.write_h5ad(f'../data/benchmarking_data/splicing_{reference_name}.h5ad')
    

In [5]:
dataset = ['mini_V3', 'mini_MN', 'mini_MD',
           'midi_NM', 'midi_Ne', 'maxi']

for ds in dataset:
    print(f'processing {ds}...')
    process_splicing(adata0, ds)
    print()

processing mini_V3...

processing mini_MN...

processing mini_MD...

processing midi_NM...

processing midi_Ne...

processing maxi...



In [6]:
print('all done!')

all done!
